In [4]:
import numpy as np 
import pandas as pd 
import requests 
import matplotlib.pyplot as plt 
from bs4 import BeautifulSoup
%matplotlib inline

In [5]:
r =requests.get('https://www.renthop.com/nyc/apartments-for-rent').content
soup =BeautifulSoup(r,'html5lib')

In [7]:
listing_divs = soup.select('div[class*=search-info]')
len(listing_divs)

20

### Pulling out the individual data points

In [24]:
href=listing_divs[0].select('a[id*=title]')[0]['href']
addy=listing_divs[0].select('a[id*=title]')[0].text
hood=listing_divs[0].select('div[id*=hood]')[0].string.replace('\n','')

### Getting other elements

In [30]:
listing_specs =listing_divs[0].select('table[id*=info] tr')
for spec in listing_specs:
    spec_data=spec.text.strip().replace(' ','_').split()
    print(spec_data)

['$2,600', '2_Bed', '1_Bath']


In [59]:
def parse_data(listing_divs):
    listing_list=[]
    for idx in range(len(listing_divs)):
        indv_listing=[]
        current_listing=listing_divs[idx]
        href=listing_divs[0].select('a[id*=title]')[0]['href']
        addy=listing_divs[0].select('a[id*=title]')[0].text.replace(',','_')
        hood=listing_divs[0].select('div[id*=hood]')[0].string.replace('\n','').replace(',','_')

        indv_listing.append(href)
        indv_listing.append(addy)
        indv_listing.append(hood)

        listing_specs=current_listing.select('table[id*=info] tr')
        for spec in listing_specs:
            try:
                indv_listing.extend(spec.text.strip().replace(' ','_').replace(',','_').split())
                indv_listing=[x for x in indv_listing if len(x.strip())!=0]
            except:
                indv_listing.extend(np.nan)
        listing_list.append(indv_listing)
    return listing_list



In [68]:
all_pages_parsed=[]
for i in range(1,21):
    target_page=f"https://www.renthop.com/search/nyc?max_price=50000&min_price={i}&sort=hopscore&q=&search=0"
    print(target_page)
    r=requests.get(target_page).content
    
    soup=BeautifulSoup(r,'html5lib')
    listing_divs=soup.select('div[class*=search-info]')
    one_page_parsed =parse_data(listing_divs)
    all_pages_parsed.extend(one_page_parsed)


https://www.renthop.com/search/nyc?max_price=50000&min_price=1&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=2&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=3&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=4&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=5&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=6&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=7&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=8&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=9&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=10&sort=hopscore&q=&search=0
https://www.renthop.com/search/nyc?max_price=50000&min_price=11&sort=hopscore&q=&search=0
https://www.renthop

In [71]:
all_pages_parsed[:3]
#df = pd.DataFrame(all_pages_parsed, columns=['url', 'address', 'neighborhood', 'rent', 'beds', 'baths','last']) 
#del df['last']
#df.head()

[['https://www.renthop.com/listings/e20-street/na/15733028',
  'E20 street',
  'Stuyvesant Town - Peter Cooper Village_ Midtown Manhattan_ Manhattan',
  '$4_831',
  '3_Bed',
  '_1_Bath'],
 ['https://www.renthop.com/listings/e20-street/na/15733028',
  'E20 street',
  'Stuyvesant Town - Peter Cooper Village_ Midtown Manhattan_ Manhattan',
  '$5_600',
  '4_Bed',
  '3_Bath'],
 ['https://www.renthop.com/listings/e20-street/na/15733028',
  'E20 street',
  'Stuyvesant Town - Peter Cooper Village_ Midtown Manhattan_ Manhattan',
  '$2_450',
  '1_Bed',
  '1_Bath']]